In [ ]:
#%cd ~ && rm -rf NAS_LLM
!git clone -b features https://github.com/WpythonW/NAS_LLM.git
!uv pip install --system "numpy<2" optuna sympy
%cd NAS_LLM/Informer2020

Cloning into 'NAS_LLM'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 286 (delta 40), reused 75 (delta 19), pack-reused 175 (from 1)
Receiving objects: 100% (286/286), 3.73 MiB | 9.41 MiB/s, done.
Resolving deltas: 100% (124/124), done.
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 212ms
Prepared 3 packages in 320ms
Uninstalled 1 package in 30ms
Installed 3 packages in 16ms
 + colorlog==6.10.1
 - numpy==2.0.2
 + numpy==1.26.4
 + optuna==4.6.0
/content/NAS_LLM/Informer2020


In [ ]:
%cd NAS_LLM/Informer2020
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
import os, optuna
import torch
import numpy as np
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
import sys
import json
from pathlib import Path
from datetime import datetime

In [ ]:
class TrialLogger:
    def __init__(self, filepath='trials.json'):
        self.filepath = Path(filepath)
        self.trials = self._load()

    def _load(self):
        return json.loads(self.filepath.read_text()) if self.filepath.exists() else []

    def _save(self):
        self.filepath.write_text(json.dumps(self.trials, indent=2))

    def add(self, trial_number, params, measures):
        self.trials.append({
            'trial': trial_number,
            'timestamp': datetime.now().isoformat(),
            **{k: float(v) for k, v in measures.items()},
            'seq_len': params['seq_len'],
            'label_len': params['label_len'],
            'e_layers': params['e_layers'],
            'n_heads': params['n_heads'],
            'factor': params['factor']
        })
        self._save()

def objective(trial):
    args = dotdict()

    # Фиксированные параметры
    args.model = 'informer'
    args.data = 'custom'
    args.root_path = './'
    args.data_path = f'../data/{dataset_name}_prepared.csv'
    args.features = 'MS'
    args.target = 'OT'
    args.freq = 'h'
    args.checkpoints = './informer_checkpoints'
    args.d_model = 512
    args.d_layers = 2
    args.d_ff = 2048
    args.dropout = 0.05
    args.attn = 'prob'
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.distil = True
    args.output_attention = False
    args.mix = True
    args.padding = 0
    args.batch_size = 32
    args.learning_rate = 0.0001
    args.loss = 'mse'
    args.lradj = 'type1'
    args.train_epochs = 6
    args.patience = 3
    args.num_workers = 0
    args.itr = 1
    args.des = 'optuna'
    args.use_amp = False

    # Используем обе GPU
    args.use_gpu = True
    args.use_multi_gpu = True
    args.devices = '0'
    args.device_ids = [0]
    args.gpu = 0

    # Перебираемые параметры с constraint
    args.seq_len = trial.suggest_categorical('seq_len', [96, 168, 336, 480, 720])
    args.label_len = trial.suggest_categorical('label_len', [24, 48, 96])

    # label_len должен быть <= seq_len
    if args.label_len > args.seq_len:
        raise optuna.TrialPruned()

    args.pred_len = pred_len ##########################CHANGE
    args.e_layers = trial.suggest_categorical('e_layers', [2, 3, 4, 6])
    args.n_heads = trial.suggest_categorical('n_heads', [8, 16])
    args.factor = trial.suggest_categorical('factor', [3, 5, 8, 10])

    args.enc_in = args.dec_in = 7
    args.c_out = 1
    args.detail_freq = args.freq
    args.freq = args.freq[-1:]

    exp = Exp_Informer(args)
    setting = f'informer_trial{trial.number}'

    exp.train(setting)
    mae_val, mse_val = exp.test(setting, flag='val')
    mae_test, mse_test = exp.test(setting, flag='test')

    measures = {'mae_val': mae_val, 'mse_val': mse_val, 'mae_test': mae_test, 'mse_test': mse_test}

    logger.add(trial.number, trial.params, measures)

    return mse_val

pred_len=168
dataset_name = 'ETTh2'
logger = TrialLogger(f'{dataset_name};pred_len={pred_len};OPTUNA.json')
storage = JournalStorage(JournalFileBackend('./optunaETTh2.log'))

study = optuna.create_study(direction='minimize',
                            storage=storage,
                            study_name=f'opt,pred_len={pred_len}',
                            load_if_exists=True)
study.optimize(objective, n_trials=35)

print(f'Best MSE: {study.best_value:.4f}')
print(f'Best params: {study.best_params}')

[I 2025-11-17 20:52:46,495] Using an existing study with name 'opt,pred_len=168' instead of creating a new one.


Use GPU: cuda:0
train 11691
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.4105996
	speed: 0.2267s/iter; left time: 473.9865s
	iters: 200, epoch: 1 | loss: 0.2385593
	speed: 0.1605s/iter; left time: 319.5319s
	iters: 300, epoch: 1 | loss: 0.2090866
	speed: 0.1615s/iter; left time: 305.3991s
Epoch: 1 cost time: 60.683987617492676
Epoch: 1, Steps: 365 | Train Loss: 0.2638041 Vali Loss: 0.6719889 Test Loss: 0.4319680
Validation loss decreased (inf --> 0.671989).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1825777
	speed: 0.3808s/iter; left time: 657.2268s
	iters: 200, epoch: 2 | loss: 0.1726370
	speed: 0.1651s/iter; left time: 268.4011s
	iters: 300, epoch: 2 | loss: 0.1816864
	speed: 0.1666s/iter; left time: 254.1594s
Epoch: 2 cost time: 60.47063732147217
Epoch: 2, Steps: 365 | Train Loss: 0.1781218 Vali Loss: 0.6831442 Test Loss: 0.4199330
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.1191210
	

[I 2025-11-17 21:00:21,043] Trial 16 finished with value: 0.43339547514915466 and parameters: {'seq_len': 336, 'label_len': 24, 'e_layers': 6, 'n_heads': 8, 'factor': 8}. Best is trial 7 with value: 0.37359756231307983.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.385198712348938, mae:0.5080816149711609
Use GPU: cuda:0
train 11691
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2229262
	speed: 0.1679s/iter; left time: 351.1079s
	iters: 200, epoch: 1 | loss: 0.2017329
	speed: 0.1692s/iter; left time: 336.7787s
	iters: 300, epoch: 1 | loss: 0.2113262
	speed: 0.1691s/iter; left time: 319.7627s
Epoch: 1 cost time: 61.71816897392273
Epoch: 1, Steps: 365 | Train Loss: 0.2634186 Vali Loss: 0.5618923 Test Loss: 0.3906305
Validation loss decreased (inf --> 0.561892).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2049111
	speed: 0.3955s/iter; left time: 682.6293s
	iters: 200, epoch: 2 | loss: 0.1712941
	speed: 0.1690s/iter; left time: 274.7324s
	iters: 300, epoch: 2 | loss: 0.1773117
	speed: 0.1693s/iter; left time: 258.2863s
Epoch: 2 cost time: 61.65203857421875
Epoch: 2, Steps: 365 | Train Loss: 0.1810438 Vali Loss: 0.5

[I 2025-11-17 21:07:53,380] Trial 17 finished with value: 0.43037810921669006 and parameters: {'seq_len': 336, 'label_len': 24, 'e_layers': 6, 'n_heads': 8, 'factor': 8}. Best is trial 7 with value: 0.37359756231307983.


...

test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.413867712020874, mae:0.5185264348983765
Use GPU: cuda:0
train 11859
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2317608
	speed: 0.1251s/iter; left time: 265.3473s
	iters: 200, epoch: 1 | loss: 0.2538676
	speed: 0.1261s/iter; left time: 254.8926s
	iters: 300, epoch: 1 | loss: 0.1961122
	speed: 0.1260s/iter; left time: 241.9741s
Epoch: 1 cost time: 46.60080528259277
Epoch: 1, Steps: 370 | Train Loss: 0.2721468 Vali Loss: 0.9676738 Test Loss: 0.4497093
Validation loss decreased (inf --> 0.967674).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1761089
	speed: 0.3039s/iter; left time: 532.1104s
	iters: 200, epoch: 2 | loss: 0.1798420
	speed: 0.1257s/iter; left time: 207.5602s
	iters: 300, epoch: 2 | loss: 0.1423884
	speed: 0.1257s/iter; left time: 194.9137s
Epoch: 2 cost time: 46.50278115272522
Epoch: 2, Steps: 370 | Train Loss: 0.1745109 Vali Loss: 0.9

[I 2025-11-17 23:34:43,402] Trial 37 finished with value: 0.8740102052688599 and parameters: {'seq_len': 168, 'label_len': 24, 'e_layers': 4, 'n_heads': 16, 'factor': 10}. Best is trial 20 with value: 0.35331326723098755.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.4330209195613861, mae:0.533315122127533
Use GPU: cuda:0
train 11547
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2891536
	speed: 0.1986s/iter; left time: 409.2666s
	iters: 200, epoch: 1 | loss: 0.2088460
	speed: 0.2000s/iter; left time: 392.1348s
	iters: 300, epoch: 1 | loss: 0.2058898
	speed: 0.1999s/iter; left time: 372.0129s
Epoch: 1 cost time: 71.94144034385681
Epoch: 1, Steps: 360 | Train Loss: 0.2590330 Vali Loss: 1.0306956 Test Loss: 0.5369738
Validation loss decreased (inf --> 1.030696).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1607451
	speed: 0.4487s/iter; left time: 763.2285s
	iters: 200, epoch: 2 | loss: 0.1707862
	speed: 0.1999s/iter; left time: 320.1147s
	iters: 300, epoch: 2 | loss: 0.1669842
	speed: 0.1999s/iter; left time: 300.0667s
Epoch: 2 cost time: 71.92709732055664
Epoch: 2, Steps: 360 | Train Loss: 0.1841177 Vali Loss: 0.7

[I 2025-11-17 23:42:00,579] Trial 38 finished with value: 0.721645176410675 and parameters: {'seq_len': 480, 'label_len': 48, 'e_layers': 6, 'n_heads': 8, 'factor': 5}. Best is trial 20 with value: 0.35331326723098755.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.5205950736999512, mae:0.595248281955719
Use GPU: cuda:0
train 11547
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2680385
	speed: 0.1781s/iter; left time: 367.0819s
	iters: 200, epoch: 1 | loss: 0.2535039
	speed: 0.1794s/iter; left time: 351.7415s
	iters: 300, epoch: 1 | loss: 0.2413520
	speed: 0.1791s/iter; left time: 333.2932s
Epoch: 1 cost time: 64.49058079719543
Epoch: 1, Steps: 360 | Train Loss: 0.2558128 Vali Loss: 0.8684095 Test Loss: 0.5128288
Validation loss decreased (inf --> 0.868409).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1739550
	speed: 0.3922s/iter; left time: 667.1500s
	iters: 200, epoch: 2 | loss: 0.1604575
	speed: 0.1791s/iter; left time: 286.7611s
	iters: 300, epoch: 2 | loss: 0.1698517
	speed: 0.1795s/iter; left time: 269.3626s
Epoch: 2 cost time: 64.48823738098145
Epoch: 2, Steps: 360 | Train Loss: 0.1748083 Vali Loss: 0.7

In [ ]:
!ls